# Lab 05: linear regressions and `pandas`
CCNY EAS 42000/A4200, Fall 2025, 2025/10/29, Prof. Spencer Hill

**SUMMARY**: We perform linear regressions using different python packages.  We also introduce the `pandas` package for tabular data analysis.

# Preliminaries

## Import needed python packages

In [6]:
from matplotlib import pyplot as plt  # for plotting
import numpy as np # for working with arrays of numerical values
import pandas as pd # for reading CSV or excel files and subsequent analyses
import scipy  # for various scientific calculations
import xarray as xr  # for loading data and subsequent analyses

## Load the Central Park data into this python session

:::{admonition} Explanation of data downloading logic (if you're interested)
:class: dropdown
To make these Jupyter notebooks work when launched to Google Colab---which you can do by clicking the "rocket" icon in the top right from the rendered version of this page on the web---we need some logic that downloads the data.

While we're at it, we use the file's "hash" to check that it has not been altered or corrupted from its original version.  We do this whether or not you've downloaded the file, since it's possible to (accidentally) modify the netCDF file on disk after you downloaded it.

In the rendered HTML version of the site, this cell is hidden, since otherwise it's a bit distracting.  But you can click on it to reveal its content.

If you're in a Google Colab session, you don't need to modify anything in that cell; just run it.  Otherwise, modify the `LOCAL_DATA_DIR` variable defined in the next python cell to point to where the dataset lives on your machine---or where you want it to be downloaded to if you don't have it already.
:::

In [7]:
!pip install pooch

# The above command installs the needed `pooch` 3rd-party package if it's not already installed.


import hashlib  # for verifying that the Central Park file is not corrupted
import pathlib  # for constructing paths to the dataset's location on disk
import sys  # for checking if this is a Google Colab session or not
import pooch  # for downloading the dataset from the web, if needed


# Replace "../data" as needed to point to the correct directory for you.
# This can be an *absolute path* or a *relative path*.  One dot, `.`, means
# "this directory", while two dots, `..`, means "go up one directory."
LOCAL_DATA_DIR = "../data"  # If you're in Colab: just ignore this.

# The URL where the dataset can be downloaded from.
DATA_URL = (
    "https://spencerahill.github.io/25f-stat-methods-course/_downloads/"
    "91803b82950d49961a65355c075439b3/central-park-station-data_1869-01-01_2023-09-30.nc"
)

# This HASH_HEX stores a "hash" which we use to verify that the data you end up
# with has not been altered or corrupted compared to the one at the above URL.
HASH_HEX = "85237a4bae1202030a36f330764fd5bd0c2c4fa484b3ae34a05db49fe7721eee"


def create_data_path(
    colab_dir="/content/data", 
    local_dir=LOCAL_DATA_DIR,
    filename="central-park-station-data_1869-01-01_2023-09-30.nc",
):
    """Set the path for the data, whether on colab or a local Jupyter session."""
    is_this_a_colab = "google.colab" in sys.modules
    if is_this_a_colab:
        data_dir = colab_dir 
    else: 
        data_dir = local_dir

    DATA_DIR = pathlib.Path(data_dir)
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    return DATA_DIR / filename


def sha256sum(path: pathlib.Path) -> str:
    """Get the hash of the file at the specified path."""
    return hashlib.sha256(path.read_bytes()).hexdigest()


DATA_PATH = create_data_path()
# Determine if we'll need to download the data, which we'll do if either (a) 
# the data can't be found, or (b) it appears corrupted/modified from the
# "master" file at the above URL.
need_fetch = (not DATA_PATH.exists()) or (sha256sum(DATA_PATH) != HASH_HEX)

# Download the data if needed.
if need_fetch:
    fetched_data = pooch.retrieve(
        url=DATA_URL, 
        known_hash=f"sha256:{HASH_HEX}",
        path=DATA_PATH.parents[0], 
        fname=DATA_PATH.name,
    )
    print(f"\nDownloaded and verified: {fetched_data}")
else:
    print(f"\nVerified existing file at {DATA_PATH}")

Looking in links: https://pypi.python.org/pypi, https://testpypi.python.org/pypi

Verified existing file at ../data/central-park-station-data_1869-01-01_2023-09-30.nc


In [8]:
import xarray as xr

# `DATA_PATH` variable was created by the hidden cell just above. 
# Un-hide that cell if you want to see the details.
ds_cp = xr.open_dataset(DATA_PATH)
ds_cp

<xarray.Dataset> Size: 5MB
Dimensions:        (time: 56520)
Coordinates:
  * time           (time) datetime64[ns] 452kB 1869-01-01 ... 2023-09-30
Data variables:
    temp_max       (time) int64 452kB ...
    temp_min       (time) int64 452kB ...
    temp_avg       (time) float64 452kB ...
    temp_anom      (time) float64 452kB ...
    heat_deg_days  (time) int64 452kB ...
    cool_deg_days  (time) int64 452kB ...
    precip         (time) float64 452kB ...
    snow_fall      (time) float64 452kB ...
    snow_depth     (time) int64 452kB ...

## Load the data again, but from an .xlsx spreadsheet file using `pandas`

## Alternative: convert the `xr.Dataset` to a `pd.Dataframe` using the `to_pandas` method

In [9]:
help(ds_cp.to_pandas)

Help on method to_pandas in module xarray.core.dataset:

to_pandas() -> 'pd.Series | pd.DataFrame' method of xarray.core.dataset.Dataset instance
    Convert this dataset into a pandas object without changing the number of dimensions.

    The type of the returned object depends on the number of Dataset
    dimensions:

    * 0D -> `pandas.Series`
    * 1D -> `pandas.DataFrame`

    Only works for Datasets with 1 or fewer dimensions.



Regarding the last line of the help message, in our case the dataset indeed has just one dimension, time, and so we can go ahead and use this method to get a `pd.DataFrame`.

In [9]:
# df is the conventional variable name used for a pandas.DataFrame
# ds is the conventional variable name used for an xarray.Dataset
df_cp = ds_cp.to_pandas()   
df_cp

,temp_max,temp_min,temp_avg,temp_anom,heat_deg_days,cool_deg_days,precip,snow_fall,snow_depth
time,,,,,,,,,
1869-01-01,29,19,24.0,-11.2,41,0,0.75,9.0,0
1869-01-02,27,21,24.0,-11.0,41,0,0.03,0.0,0
1869-01-03,35,27,31.0,-3.8,34,0,0.00,0.0,0
1869-01-04,37,34,35.5,0.8,29,0,0.18,0.0,0
1869-01-05,43,37,40.0,5.5,25,0,0.05,0.0,0
...,...,...,...,...,...,...,...,...,...
2023-09-26,60,54,57.0,-8.5,8,0,0.20,0.0,0
2023-09-27,64,50,57.0,-8.1,8,0,0.00,0.0,0
2023-09-28,65,56,60.5,-4.2,4,0,0.38,0.0,0


# Some basics on pandas

## Basic usage and terminology

The standard way of importing it is to abbreviate it to `pd`: `import pandas as pd`.

Typically, `pd.Series` objects are used to represent a single, 1-D variable.

Typically, `pd.DataFrame` objects are used to represent *tabular* data: either one 2-D variable or multiple, 1-D variables all sharing the same coordinate values along that dimension.

In our case, the dataframe is a collection of nine 1D timeseries.

Each column in a `pd.DataFrame` object is itself a `pd.Series`---analogous to how each variable in an `xr.Dataset` is itself an `xr.DataArray`.

## Accessing variables

For more, see the [pandas doc page on indexing and selecting data](https://pandas.pydata.org/docs/user_guide/indexing.html).

Use square brackets and the name of the column to grab that variable:

In [10]:
print(type(df_cp["temp_max"]))
df_cp["temp_max"]

<class 'pandas.core.series.Series'>


time
1869-01-01    29
1869-01-02    27
1869-01-03    35
1869-01-04    37
1869-01-05    43
              ..
2023-09-26    60
2023-09-27    64
2023-09-28    65
2023-09-29    63
2023-09-30    66
Name: temp_max, Length: 56520, dtype: int64

To grab an individual row, there are two ways, `loc` and `iloc`.  **Important**: unlike normal methods and functions, `loc` and `iloc` don't use parentheses.  Instead, they use square brackets.  (It's because they're technically something called an "Accessor" rather than being methods.)

**loc** is for *label-based* indexing.  Give it the actual value of the index you want to grab.  In this case, because the Index is a datetime object, you can specify the date as a string:

In [11]:
df_cp.loc["1989-04-22"]

temp_max         55.0
temp_min         36.0
temp_avg         45.5
temp_anom       -10.7
heat_deg_days    19.0
cool_deg_days     0.0
precip            0.0
snow_fall         0.0
snow_depth        0.0
Name: 1989-04-22 00:00:00, dtype: float64

You can also use *slicing* to grab multiple values:

In [13]:
df_cp.loc["1989-04-22":"1989-04-24"]

,temp_max,temp_min,temp_avg,temp_anom,heat_deg_days,cool_deg_days,precip,snow_fall,snow_depth
time,,,,,,,,,
1989-04-22,55,36,45.5,-10.7,19,0,0.0,0.0,0
1989-04-23,58,34,46.0,-10.5,19,0,0.0,0.0,0
1989-04-24,62,37,49.5,-7.4,15,0,0.0,0.0,0


Notice that it includes *both endpoints*.

You can also use `iloc`, which grabs things by *integer index*.  That means, `0` is the 1st row of data, `1` the second row, etc.:

In [17]:
df_cp.iloc[0]

temp_max         29.00
temp_min         19.00
temp_avg         24.00
temp_anom       -11.20
heat_deg_days    41.00
cool_deg_days     0.00
precip            0.75
snow_fall         9.00
snow_depth        0.00
Name: 1869-01-01 00:00:00, dtype: float64

Just like for `.loc` you can use slicing:

In [18]:
df_cp.iloc[1000:1002]

,temp_max,temp_min,temp_avg,temp_anom,heat_deg_days,cool_deg_days,precip,snow_fall,snow_depth
time,,,,,,,,,
1871-09-28,60,47,53.5,-11.2,11,0,0.0,0.0,0
1871-09-29,56,44,50.0,-14.3,15,0,0.0,0.0,0


Notice that, unlike for `.loc`, it does **not** include the right endpoint.  (Which is the same behavior as for indexing lists, numpy arrays, and other things too.)

# Compute the linear regression

### Using `scipy.stats.linregress`

In [10]:
help(scipy.stats.linregress)

Help on function linregress in module scipy.stats._stats_py:

linregress(
    x,
    y,
    alternative='two-sided',
    *,
    axis=0,
    nan_policy='propagate',
    keepdims=False
)
    Calculate a linear least-squares regression for two sets of measurements.

    Parameters
    ----------
    x, y : array_like
        Two sets of measurements.  Both arrays should have the same length N.
    alternative : {'two-sided', 'less', 'greater'}, optional
        Defines the alternative hypothesis. Default is 'two-sided'.
        The following options are available:

        * 'two-sided': the slope of the regression line is nonzero
        * 'less': the slope of the regression line is less than zero
        * 'greater':  the slope of the regression line is greater than zero

        .. versionadded:: 1.7.0
    axis : int or None, default: 0
        If an int, the axis of the input along which to compute the statistic.
        The statistic of each axis-slice (e.g. row) of the input will ap

The inputs to `scipy.stats.linregress` can be `pd.Series`, `xr.DataArray`, numpy arrays, or even plain old lists:

In [27]:
# nan = Not a Number, for missing values
np.any(np.isnan(df_cp["snow_fall"]))

np.True_

In [28]:
scipy.stats.linregress(df_cp["temp_max"], df_cp["snow_fall"])

LinregressResult(slope=np.float64(nan), intercept=np.float64(nan), rvalue=np.float64(nan), pvalue=np.float64(nan), stderr=np.float64(nan), intercept_stderr=np.float64(nan))

In [33]:
# change nan_policy
lr_snow_vs_tmax = scipy.stats.linregress(
    ds_cp["temp_max"], ds_cp["snow_fall"], nan_policy="omit")
lr_snow_vs_tmax

LinregressResult(slope=np.float64(-0.006973015589052081), intercept=np.float64(0.5160349776366013), rvalue=np.float64(-0.19627996789271956), pvalue=np.float64(0.0), stderr=np.float64(0.00014960900816769152), intercept_stderr=np.float64(0.009738693235191215))

In [34]:
lr_snow_vs_tmax.slope

np.float64(-0.006973015589052081)

In [35]:
lr_snow_vs_tmax.intercept

np.float64(0.5160349776366013)

In [37]:
type(ds_cp["snow_depth"].values)

numpy.ndarray

In [36]:
scipy.stats.linregress(ds_cp["snow_depth"].values, ds_cp["temp_max"].values)

LinregressResult(slope=np.float64(-3.598971141084667), intercept=np.float64(62.157608768059596), rvalue=np.float64(-0.233427342412877), pvalue=np.float64(0.0), stderr=np.float64(0.06306184810935893), intercept_stderr=np.float64(0.07799165142915197))

In [36]:
scipy.stats.linregress(ds_cp["snow_depth"].values, ds_cp["temp_max"].values)

LinregressResult(slope=np.float64(-3.598971141084667), intercept=np.float64(62.157608768059596), rvalue=np.float64(-0.233427342412877), pvalue=np.float64(0.0), stderr=np.float64(0.06306184810935893), intercept_stderr=np.float64(0.07799165142915197))

In [37]:
scipy.stats.linregress(list(ds_cp["snow_depth"].values), list(ds_cp["temp_max"].values))

LinregressResult(slope=np.float64(-3.598971141084667), intercept=np.float64(62.157608768059596), rvalue=np.float64(-0.233427342412877), pvalue=np.float64(0.0), stderr=np.float64(0.06306184810935893), intercept_stderr=np.float64(0.07799165142915197))

# Performing linear regressions

## HW question 1: block max P vs. MJJAS mean T

In [48]:
df_cp.index.year

Index([1869, 1869, 1869, 1869, 1869, 1869, 1869, 1869, 1869, 1869,
       ...
       2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023],
      dtype='int32', name='time', length=56520)

In [49]:
# in pandas for learning purposes
# get block max P
# in xarray, this would be ds_cp["precip"].groupby("time.year").max()
df_cp["precip"].groupby(df_cp.index.year).max()

time
1869    2.60
1870    1.91
1871    2.70
1872    2.91
1873    4.80
        ... 
2019    1.83
2020    2.54
2021    7.13
2022    1.85
2023    5.48
Name: precip, Length: 155, dtype: float64